In [5]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Configuration
DATA_PATH = "generated_images_8/"  # Directory with 0/ and 1/ subdirectories
MODEL_PATH ="best_model_des.pth"  # Path to your saved model
IMG_SIZE = (256, 256)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the trained model
class DenseCNN(nn.Module):
    def __init__(self, base_channels=32):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, base_channels, 3, padding=1),
            nn.BatchNorm2d(base_channels),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(base_channels, base_channels*2, 3, padding=1),
            nn.BatchNorm2d(base_channels*2),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(base_channels*2, base_channels*4, 3, padding=1),
            nn.BatchNorm2d(base_channels*4),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(base_channels*4, base_channels*8, 3, padding=1),
            nn.BatchNorm2d(base_channels*8),
            nn.ReLU(),
        )
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Dropout(0.2),
            nn.Linear(base_channels*8, 2)
        )
        
    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

model = DenseCNN().to(device)
model.load_state_dict(torch.load(MODEL_PATH))
model.eval()

# Image preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

def evaluate_model(directory_path):
    """Evaluate model on images in directory structure"""
    true_labels = []
    pred_labels = []
    confidences = []
    image_paths = []
    
    # Walk through directory structure
    for label in ['0', '1']:
        class_dir = os.path.join(directory_path, label)
        if not os.path.exists(class_dir):
            continue
            
        for img_file in os.listdir(class_dir):
            if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(class_dir, img_file)
                
                # Load and preprocess image
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    continue
                    
                img = cv2.resize(img, IMG_SIZE)
                img_tensor = transform(Image.fromarray(img)).unsqueeze(0).to(device)
                
                # Get prediction
                with torch.no_grad():
                    output = model(img_tensor)
                    probabilities = torch.softmax(output, dim=1)
                    _, predicted = torch.max(output, 1)
                    confidence = probabilities[0][predicted].item()
                
                # Store results
                true_labels.append(int(label))
                pred_labels.append(predicted.item())
                confidences.append(confidence)
                image_paths.append(img_path)
    
    # Calculate metrics
    accuracy = accuracy_score(true_labels, pred_labels)
    print(f"\nEvaluation Results for {directory_path}:")
    print(f"Accuracy: {accuracy:.4f}")
    print("\nClassification Report:")
    print(classification_report(true_labels, pred_labels, target_names=['Benign (0)', 'Malignant (1)']))
    print("\nConfusion Matrix:")
    print(confusion_matrix(true_labels, pred_labels))
    
    # Print some examples
    print("\nSample Predictions:")
    for i in range(min(5, len(image_paths))):
        print(f"Image: {os.path.basename(image_paths[i])} | "
              f"True: {true_labels[i]} | "
              f"Pred: {pred_labels[i]} | "
              f"Confidence: {confidences[i]:.2f}")
    
    return {
        'accuracy': accuracy,
        'true_labels': true_labels,
        'pred_labels': pred_labels,
        'confidences': confidences,
        'image_paths': image_paths
    }

if __name__ == "__main__":
    print(f"Evaluating model on images in: {DATA_PATH}")
    print(f"Using device: {device}")
    results = evaluate_model(DATA_PATH)

Evaluating model on images in: generated_images_8/
Using device: cuda

Evaluation Results for generated_images_8/:
Accuracy: 0.9300

Classification Report:
               precision    recall  f1-score   support

   Benign (0)       0.92      0.94      0.93        50
Malignant (1)       0.94      0.92      0.93        50

     accuracy                           0.93       100
    macro avg       0.93      0.93      0.93       100
 weighted avg       0.93      0.93      0.93       100


Confusion Matrix:
[[47  3]
 [ 4 46]]

Sample Predictions:
Image: generated7_0.png | True: 0 | Pred: 0 | Confidence: 0.96
Image: generated7_1.png | True: 0 | Pred: 0 | Confidence: 0.94
Image: generated7_10.png | True: 0 | Pred: 0 | Confidence: 0.84
Image: generated7_11.png | True: 0 | Pred: 0 | Confidence: 0.92
Image: generated7_12.png | True: 0 | Pred: 0 | Confidence: 0.75


In [ ]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Configuration
DATA_PATH = "generated_images_1/"  # Directory with 0/ and 1/ subdirectories
MODEL_PATH = "best_model_des.pth"  # Updated to match trained model
IMG_SIZE = (256, 256)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Updated Model Architecture to match training
class DenseCNN(nn.Module):
    def __init__(self, base_channels=64):  # Increased base channels
        super().__init__()
        
        # Initial convolution block
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, base_channels, 3, padding=1),
            nn.BatchNorm2d(base_channels),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        # Dense block 1
        self.conv2 = nn.Sequential(
            nn.Conv2d(base_channels, base_channels*2, 3, padding=1),
            nn.BatchNorm2d(base_channels*2),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        # Dense block 2 with residual connection
        self.conv3 = nn.Sequential(
            nn.Conv2d(base_channels*2, base_channels*4, 3, padding=1),
            nn.BatchNorm2d(base_channels*4),
            nn.ReLU(),
            nn.Conv2d(base_channels*4, base_channels*2, 3, padding=1),  # Match input channels
            nn.BatchNorm2d(base_channels*2),
            nn.ReLU(),
        )
        self.downsample3 = nn.Conv2d(base_channels*2, base_channels*2, 1)  # Identity mapping
        
        # Dense block 3 with residual connection
        self.conv4 = nn.Sequential(
            nn.Conv2d(base_channels*2, base_channels*8, 3, padding=1),
            nn.BatchNorm2d(base_channels*8),
            nn.ReLU(),
            nn.Conv2d(base_channels*8, base_channels*2, 3, padding=1),  # Match input channels
            nn.BatchNorm2d(base_channels*2),
            nn.ReLU(),
        )
        self.downsample4 = nn.Conv2d(base_channels*2, base_channels*2, 1)  # Identity mapping
        
        # Attention mechanism
        self.attention = nn.Sequential(
            nn.Conv2d(base_channels*2, 1, 1),
            nn.Sigmoid()
        )
        
        # Final pooling and classifier
        self.final_pool = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(base_channels*2, base_channels),
            nn.BatchNorm1d(base_channels),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(base_channels, 2)
        )
        
    def forward(self, x):
        x1 = self.conv1(x)  # 256x256 -> 128x128
        x2 = self.conv2(x1)  # 128x128 -> 64x64
        
        # First residual block
        identity = x2
        x3 = self.conv3(x2)
        x3 = x3 + self.downsample3(identity)
        
        # Second residual block
        identity = x3
        x4 = self.conv4(x3)
        x4 = x4 + self.downsample4(identity)
        
        # Attention mechanism
        attention = self.attention(x4)
        x4 = x4 * attention
        
        # Final classification
        x4 = self.final_pool(x4)
        return self.classifier(x4)

# Load the trained model
model = DenseCNN().to(device)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()

# Image preprocessing (should match training)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

def evaluate_model(directory_path):
    """Evaluate model on images in directory structure"""
    true_labels = []
    pred_labels = []
    confidences = []
    image_paths = []
    
    # Walk through directory structure
    for label in ['0', '1']:
        class_dir = os.path.join(directory_path, label)  # Updated to look in rf subdirectory
        
            
        for img_file in os.listdir(class_dir):
            if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(class_dir, img_file)
                
                # Load and preprocess image
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    print(f"Warning: Could not read image {img_path}")
                    continue
                    
                img = cv2.resize(img, IMG_SIZE)
                img_tensor = transform(Image.fromarray(img)).unsqueeze(0).to(device)
                
                # Get prediction
                with torch.no_grad():
                    output = model(img_tensor)
                    probabilities = torch.softmax(output, dim=1)
                    _, predicted = torch.max(output, 1)
                    confidence = probabilities[0][predicted].item()
                
                # Store results
                true_labels.append(int(label))
                pred_labels.append(predicted.item())
                confidences.append(confidence)
                image_paths.append(img_path)
    
    if not true_labels:
        print("No images found for evaluation")
        return None
    
    # Calculate metrics
    accuracy = accuracy_score(true_labels, pred_labels)
    print(f"\nEvaluation Results for {directory_path}:")
    print(f"Accuracy: {accuracy:.4f}")
    print("\nClassification Report:")
    print(classification_report(true_labels, pred_labels, target_names=['Benign (0)', 'Malignant (1)']))
    print("\nConfusion Matrix:")
    print(confusion_matrix(true_labels, pred_labels))
    
    # Print some examples
    print("\nSample Predictions:")
    for i in range(min(5, len(image_paths))):
        print(f"Image: {os.path.basename(image_paths[i])} | "
              f"True: {true_labels[i]} | "
              f"Pred: {pred_labels[i]} | "
              f"Confidence: {confidences[i]:.2f}")
    
    return {
        'accuracy': accuracy,
        'true_labels': true_labels,
        'pred_labels': pred_labels,
        'confidences': confidences,
        'image_paths': image_paths
    }

if __name__ == "__main__":
    print(f"Evaluating model on images in: {DATA_PATH}")
    print(f"Using device: {device}")
    print(f"Loading model from: {MODEL_PATH}")
    
    results = evaluate_model(DATA_PATH)
    
    if results:
        print("\nEvaluation completed successfully")
    else:
        print("\nEvaluation failed - no valid images found")

/tmp/ipykernel_203261/45189740.py:100: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_PATH, map_location=device))


Evaluating model on images in: generated_images_1/
Using device: cuda
Loading model from: best_model_full_data.pth

Evaluation Results for generated_images_1/:
Accuracy: 0.7000

Classification Report:
               precision    recall  f1-score   support

   Benign (0)       0.64      0.92      0.75        25
Malignant (1)       0.86      0.48      0.62        25

     accuracy                           0.70        50
    macro avg       0.75      0.70      0.68        50
 weighted avg       0.75      0.70      0.68        50


Confusion Matrix:
[[23  2]
 [13 12]]

Sample Predictions:
Image: generated_0.png | True: 0 | Pred: 0 | Confidence: 0.86
Image: generated_1.png | True: 0 | Pred: 0 | Confidence: 0.83
Image: generated_10.png | True: 0 | Pred: 0 | Confidence: 0.98
Image: generated_11.png | True: 0 | Pred: 0 | Confidence: 0.94
Image: generated_12.png | True: 0 | Pred: 0 | Confidence: 0.67

Evaluation completed successfully
